<a href="https://colab.research.google.com/github/SebastienLiger/Portfolio-Finance/blob/Finance/Pricing_Options_Black_Scholes_DELTA_AIR_LINES.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import yfinance as yf
import numpy as np
import pandas as pd
from scipy.stats import norm
from tabulate import tabulate

In [6]:
# Fonctions du modèle Black-Scholes
def black_scholes(S, K, T, r, sigma):
    """
    Calcule les prix d'une option call et put selon le modèle Black-Scholes.
    """
    d1, d2 = calculate_d1_d2(S, K, T, r, sigma)
    call = S * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)
    put = K * np.exp(-r * T) * norm.cdf(-d2) - S * norm.cdf(-d1)
    return call, put

def calculate_d1_d2(S, K, T, r, sigma):
    """
    Calcule d1 et d2 pour le modèle Black-Scholes.
    """
    d1 = (np.log(S / K) + (r + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    return d1, d2

def black_scholes_greeks(S, K, T, r, sigma, d1, d2):
    """
    Calcule les Greeks pour une option selon le modèle Black-Scholes.
    """
    call_delta = norm.cdf(d1)
    put_delta = norm.cdf(-d1)
    gamma = norm.pdf(d1) / (S * sigma * np.sqrt(T))
    call_theta = (-S * sigma * norm.pdf(d1)) / (2 * np.sqrt(T)) - r * K * np.exp(-r * T) * norm.cdf(d2)
    put_theta = (-S * sigma * norm.pdf(-d1)) / (2 * np.sqrt(T)) + r * K * np.exp(-r * T) * norm.cdf(-d2)
    vega = S * np.sqrt(T) * norm.pdf(d1)
    rho_call = K * T * np.exp(-r * T) * norm.cdf(d2)
    rho_put = -K * T * np.exp(-r * T) * norm.cdf(-d2)
    return {'call_delta': call_delta, 'put_delta': put_delta, 'gamma': gamma,
            'call_theta': call_theta, 'put_theta': put_theta, 'vega': vega,
            'rho_call': rho_call, 'rho_put': rho_put}


In [7]:
from datetime import datetime, timedelta

# Créez une instance de l'objet Ticker pour l'action Delta Airlines (DAL)
delta_airlines = yf.Ticker("OYC.F")

# Obtenez les données actuelles
current_data = delta_airlines.history(period="1d")

# Extrayez le prix actuel
current_price = current_data["Close"].iloc[0]

# Affichez le prix actuel
print(f"Prix : €{current_price:.2f}")

# Obtention des données historiques sur la dernière année
historical_data = delta_airlines.history(period="1y")

# Calcul de la volatilité sur la dernière année
historical_data["Log Returns"] = np.log(historical_data["Close"] / historical_data["Close"].shift(1))
volatility = historical_data["Log Returns"].std() * np.sqrt(252)  # 252 jours ouvrables dans une année

print(f"Volatilité : {volatility * 100:.2f}%")

# Fonction pour trouver le dernier vendredi
def last_friday():
    today = datetime.today()
    last_friday = today - timedelta(days=(today.weekday() - 4) % 7 + 1)  # Ajuste au dernier vendredi
    return last_friday.date()

# Obtenez la date du dernier vendredi
date_last_friday = last_friday()

# Créez l'objet Ticker pour ^TYX
treasuries_us = yf.Ticker("^TYX")

# Récupérer les données pour le dernier vendredi
current_data_risk_free_rate = treasuries_us.history(start=date_last_friday, end=date_last_friday + timedelta(days=1))

# Vérifiez si des données de prix sont disponibles et récupérez la dernière donnée non nulle
if not current_data_risk_free_rate.empty and not current_data_risk_free_rate["Close"].dropna().empty:
    current_price_risk_free_rate = current_data_risk_free_rate["Close"].dropna().iloc[-1]
else:
    # Si aucune donnée n'est disponible, définir une valeur par défaut ou gérer l'erreur
    print("Aucune donnée de taux sans risque disponible pour le dernier vendredi.")
    # Utiliser une valeur de taux sans risque par défaut
    current_price_risk_free_rate = 0.03

# Calculer le taux sans risque actuel
current_price_RFR = current_price_risk_free_rate / 100

print(f"Risk free rate for last Friday ({date_last_friday}): {current_price_RFR:.4f}")

Prix : €39.99
Volatilité : 29.87%
Risk free rate for last Friday (2024-03-07): 0.0425


In [8]:
# Paramètres de l'option
S, K, T, r, sigma = current_price, 45, 1, current_price_RFR, volatility

# Calcul de d1, d2, prix des options et Greeks
d1, d2 = calculate_d1_d2(S, K, T, r, sigma)
call_price, put_price = black_scholes(S, K, T, r, sigma)
greeks = black_scholes_greeks(S, K, T, r, sigma, d1, d2)

# Création du DataFrame
data = {
    "Call": [f"{call_price:.2f}€", f"{greeks['call_delta']:.2f}", f"{greeks['gamma']:.2f}", f"{greeks['call_theta']:.2f}", f"{greeks['vega']:.2f}", f"{greeks['rho_call']:.2f}", f"{d1:.2f}", f"{d2:.2f}"],
    "Put": [f"{put_price:.2f}€", f"{greeks['put_delta']:.2f}", f"{greeks['gamma']:.2f}", f"{greeks['put_theta']:.2f}", f"{greeks['vega']:.2f}", f"{greeks['rho_put']:.2f}", f"{d1:.2f}", f"{d2:.2f}"]
}
index = ["Price", "Delta", "Gamma", "Theta", "Vega", "Rho", "d1", "d2"]
df = pd.DataFrame(data, index=index)

# Affichage du tableau
print(tabulate(df, headers='keys', tablefmt='fancy_grid'))

╒═══════╤════════╤════════╕
│       │ Call   │ Put    │
╞═══════╪════════╪════════╡
│ Price │ 3.52€  │ 6.66€  │
├───────┼────────┼────────┤
│ Delta │ 0.46   │ 0.54   │
├───────┼────────┼────────┤
│ Gamma │ 0.03   │ 0.03   │
├───────┼────────┼────────┤
│ Theta │ -3.00  │ -1.17  │
├───────┼────────┼────────┤
│ Vega  │ 15.87  │ 15.87  │
├───────┼────────┼────────┤
│ Rho   │ 14.83  │ -28.30 │
├───────┼────────┼────────┤
│ d1    │ -0.10  │ -0.10  │
├───────┼────────┼────────┤
│ d2    │ -0.40  │ -0.40  │
╘═══════╧════════╧════════╛
